Trying out: https://www.kaggle.com/tunguz/logistic-regression-with-words-and-char-n-grams/code

In [1]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack

In [2]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

#importing the dataset
train = pd.read_csv("../data/train.csv").fillna(' ')
test = pd.read_csv("../data/test.csv").fillna(' ')
                                             
train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

In [4]:
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)
word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)

In [3]:
char_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='char',
    stop_words='english',
    ngram_range=(2, 6),
    max_features=50000)
char_vectorizer.fit(all_text)
train_char_features = char_vectorizer.transform(train_text)
test_char_features = char_vectorizer.transform(test_text)

In [8]:
bigram_vectorizer = TfidfVectorizer(
    max_features=10000,
    strip_accents='unicode', 
    analyzer='word',
    ngram_range=(1,2),
    use_idf=1,
    smooth_idf=True,
    sublinear_tf=False,
    stop_words = 'english', 
    dtype=np.float32)

bigram_vectorizer.fit(all_text)
train_bigram_features = bigram_vectorizer.transform(train_text)
test_bigram_features = bigram_vectorizer.transform(test_text)

In [6]:
import pickle

pickle.dump(train_char_features, open('../models/train_char_vectors.pkl', 'wb'))
pickle.dump(test_char_features, open('../models/test_char_vectors.pkl', 'wb'))

#bst = pickle.load( open('./models/char_vectors.pkl', 'rb') )

In [9]:
train_features = hstack([train_char_features, train_word_features, train_bigram_features])
test_features = hstack([test_char_features, test_word_features, test_bigram_features])

In [10]:
scores = []
submission = pd.DataFrame.from_dict({'id': test['id']})
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(solver='sag')

    cv_scoMre = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    scores.append(cv_score)
    print('CV score for class {} is {}'.format(class_name, cv_score))

    classifier.fit(train_features, train_target)
    submission[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(scores)))

submission.to_csv('../submissions/toxic_lr_char_tdidf.csv', index=False)

CV score for class toxic is 0.9777000009616527
CV score for class severe_toxic is 0.9879403764538561
CV score for class obscene is 0.9894913431495848
CV score for class threat is 0.9895776813298989
CV score for class insult is 0.9817723622500912
CV score for class identity_hate is 0.980953238198401
Total CV score is 0.9845725003905809


### Records

* CV score for class toxic is 0.9777000009616527
* CV score for class severe_toxic is 0.9879403764538561
* CV score for class obscene is 0.9894913431495848
* CV score for class threat is 0.9895776813298989
* CV score for class insult is 0.9817723622500912
* CV score for class identity_hate is 0.980953238198401
* Total CV score is 0.9845725003905809